In [ ]:
Note
In the District Summary more students passed reading than math by 10%.  
Charter Schools have significantly higher reading and math scores.
The math and readings scores do not fluctuate more than 2 points from 9th to 12th grade.
Medium size schools have the highest math and overall passing grade scores.

Note
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File 
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# compute district stats
total_schools = len(school_data_complete["school_name"].unique())
total_students = len(school_data_complete["Student ID"])

total_budget = float(school_data["budget"].sum())

avg_math_score = round(float(school_data_complete["math_score"].mean()), 2)
avg_reading_score = round(float(school_data_complete["reading_score"].mean()), 2)

num_math_passing = len(school_data_complete.loc[school_data_complete["math_score"] >= 70]["Student ID"])
percent_math_passing = float(round(num_math_passing / total_students * 100, 2))
num_reading_passing = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70]["Student ID"])
percent_reading_passing = float(round(num_reading_passing / total_students * 100, 2))
overall_passing_rate = round((percent_reading_passing + percent_math_passing) / 2, 2)

# create new dataframe
district_summary = pd.DataFrame({"Total Schools":total_schools, "Total Students":total_students,
                                "Total Budget":total_budget, "Average Math Score":avg_math_score,
                                "Average Reading Score":avg_reading_score, 
                                "Percent Passed Math":percent_math_passing,
                                "Percent Passed Reading":percent_reading_passing,
                                "Overall Passing Rate":overall_passing_rate}, index=[0])

# formatting
district_summary["Total Students"] = district_summary["Total Students"].map('{:,}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,}'.format)
district_summary["Percent Passed Math"] = district_summary["Percent Passed Math"].map('{}%'.format)
district_summary["Percent Passed Reading"] = district_summary["Percent Passed Reading"].map('{}%'.format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map('{}%'.format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passed Math,Percent Passed Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.0",78.99,81.88,74.98%,85.81%,80.4%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [7]:
# Grouped our complete data frame by school name 
grouped_school = school_data_complete.groupby(['school_name'])
# Calculate the total student for each school
total_student = grouped_school.size()
# Get the school type for each school
school_type = grouped_school['type'].first()
# Calculate the total budget for each school
total_budget = grouped_school['budget'].first()
# Calculate the budget per student for each school
t_budget_per_student = total_budget/total_student
# Calculate the average math score for each school
average_math_score = grouped_school['math_score'].mean()
# Calculate the average reading score for each school
average_reading_score = grouped_school['reading_score'].mean()
# Calculate the percentange of passing math score for each school
grouped_passing_math = school_data_complete[school_data_complete['math_score']>=70].groupby(['school_name']).size()
percent_passing_math = (grouped_passing_math/total_student)*100
# Calculate the percentange of passing math score for each school
grouped_passing_reading = school_data_complete[school_data_complete['reading_score']>=70].groupby(['school_name']).size()
percent_passing_reading = (grouped_passing_reading/total_student)*100
# Calculate the overall passing score for each school
percent_overall_passing = (percent_passing_math + percent_passing_reading)/2

# Create a dataframe to hold the above results
school={
    'School Type': school_type,
    'Total Students':total_student,
    'Total School Budget': total_budget,
    'Per Student Budget': t_budget_per_student,
    'Average Math Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Math': percent_passing_math,
    '% Passing Reading': percent_passing_reading,
    '% Overall Passing Rate': percent_overall_passing,
}
school_summary = pd.DataFrame(school)
# Create a copy of school summary data frame before formatting to be able to use the numeric data on original data frame later
displayed_school_summary = school_summary.copy()
# Formatting the display data frame
displayed_school_summary['Per Student Budget'] = displayed_school_summary['Per Student Budget'].map('${:,.2f}'.format)
displayed_school_summary['Total School Budget'] = displayed_school_summary['Total School Budget'].map('${:,.2f}'.format)
displayed_school_summary.index.name = None

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
grouped_schools = school_data_complete.groupby("school_name")

# add data that's easier to get from school_data
school_summary = pd.DataFrame({"School Name":school_data["school_name"], "School Type":school_data["type"],
                             "Total Budget":school_data["budget"]})
school_summary = school_summary.set_index("School Name")
school_summary = school_summary.sort_values("School Name")

# do calculations on data from groupby object
school_summary["Total Students"] = grouped_schools["Student ID"].count()
school_summary["Budget Per Student"] = school_summary["Total Budget"] / school_summary["Total Students"]
school_summary["Average Math Score"] = round(grouped_schools["math_score"].mean(), 2)
school_summary["Average Reading Score"] = round(grouped_schools["reading_score"].mean(), 2)

# calculate percentages of students passing using apply
school_summary["Percent Passing Math"] = round((grouped_schools.apply(lambda x: (x["math_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 2)
school_summary["Percent Passing Reading"] = round((grouped_schools.apply(lambda x: (x["reading_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 2)
school_summary["Overall Passing Rate"] = round((school_summary["Percent Passing Math"] + 
                                               school_summary["Percent Passing Reading"]) / 2, 2)

school_summary = school_summary.sort_values("Overall Passing Rate", ascending=False)

# add formatting
school_summary["Total Budget"] = school_summary["Total Budget"].map('${:,}'.format)

# could also format the following columns, but we need the values for analyses later

# school_summary["Percent Passing Math"] = school_summary["Percent Passing Math"].map('{}%'.format)
# school_summary["Percent Passing Reading"] = school_summary["Percent Passing Reading"].map('{}%'.format)
# school_summary["Overall Passing Rate"] = school_summary["Overall Passing Rate"].map('{}%'.format)
# school_summary["Budget Per Student"] = school_summary["Budget Per Student"].map('${:}'.format)
# school_summary["Total Students"] = school_summary["Total Students"].map('{:,}'.format)

school_summary.head()

,School Type,Total Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356",1858,582.0,83.06,83.98,94.13,97.04,95.58
Thomas High School,Charter,"$1,043,130",1635,638.0,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,"$585,858",962,609.0,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,"$917,500",1468,625.0,83.35,83.82,93.39,97.14,95.26
Wilson High School,Charter,"$1,319,574",2283,578.0,83.27,83.99,93.87,96.54,95.21


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [8]:
# Sort and display the five worst-performing schools
worst_performing_schools = displayed_school_summary.sort_values(by='% Overall Passing Rate')
worst_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [9]:
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"]
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"]

math_ninth = ninth.groupby("school_name")["math_score"].mean()
math_tenth = tenth.groupby("school_name")["math_score"].mean()
math_eleventh = eleventh.groupby("school_name")["math_score"].mean()
math_twelfth = twelfth.groupby("school_name")["math_score"].mean()

math_grades = pd.DataFrame({"9th":math_ninth, "10th":math_tenth, 
                             "11th":math_eleventh, "12th":math_twelfth})

math_grades = round(math_grades, 4)

math_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.0837,76.9968,77.5156,76.4922
Cabrera High School,83.0947,83.1545,82.7656,83.2775
Figueroa High School,76.4030,76.5400,76.8843,77.1514
Ford High School,77.3613,77.6723,76.9181,76.1800
Griffin High School,82.0440,84.2291,83.8421,83.3562
Hernandez High School,77.4385,77.3374,77.1360,77.1866
Holden High School,83.7874,83.4298,85.0000,82.8554
Huang High School,77.0273,75.9087,76.4466,77.2256
Johnson High School,77.1879,76.6911,77.4917,76.8632


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
# Calculate the average reading score for students of 9th grade at each school
school_avg_reading_9th = school_data_complete[school_data_complete['grade']=='9th'].groupby('school_name')['reading_score'].mean()
# Calculate the average reading score for students of 10th grade at each school
school_avg_reading_10th = school_data_complete[school_data_complete['grade']=='10th'].groupby('school_name')['reading_score'].mean()
# Calculate the average reading score for students of 11th grade at each school
school_avg_reading_11th = school_data_complete[school_data_complete['grade']=='11th'].groupby('school_name')['reading_score'].mean()
# Calculate the average reading score for students of 12th grade at each school
school_avg_reading_12th = school_data_complete[school_data_complete['grade']=='12th'].groupby('school_name')['reading_score'].mean()

# Create a dataframe to hold the above results
grade_reading_score={
    '9th':school_avg_reading_9th,
    '10th':school_avg_reading_10th,
    '11th':school_avg_reading_11th,
    '12th':school_avg_reading_12th,
    }

reading_score_by_grade = pd.DataFrame(grade_reading_score)
reading_score_by_grade.index.name = None
reading_score_by_grade.head(20)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
# Sample bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [12]:
# Create a new data frame by locating the desired columns
scores_spending = school_summary.loc[:,['Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Add a new columns named Spending Ranges (Per Student) and binning based off budget per student
scores_spending['Spending Ranges (Per Student)']= pd.cut(school_summary['Per Student Budget'],spending_bins,labels=group_names)
# Create a group based off of the bins
scores_spending = scores_spending.groupby('Spending Ranges (Per Student)').mean()
scores_spending.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# Sample bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [15]:
# Create a new data frame by locating the desired columns
scores_size = school_summary.loc[:,['Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Add a new columns named School Size and binning based off total students
scores_size['School Size']= pd.cut(school_summary['Total Students'],size_bins,labels=group_names)
# Create a group based off of the bins
scores_size = scores_size.groupby('School Size').mean()
scores_size.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type

In [17]:
# Create a new data frame with our desired columns
scores_type = school_summary[['School Type','Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Create a group based off of the school type
scores_type = scores_type.groupby('School Type').mean()
scores_type.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
